In [75]:
import pandas as pd
import numpy as np
import os
import string

### Read File

In [76]:
df = pd.read_csv("data/outpatient_sample.csv")

### Count Number of Visits

In [77]:
total_visits = df.groupby('Patient_ID').size().reset_index(name='Total_Visits')

### Get unique Physicians for each Patient

In [78]:
def unique_physicians(group):
    """
    Collects all unique physician IDs (Primary, Operating, Other) for a given patient
    across all their visits within the group.

    Args:
        group (pd.DataFrame): A DataFrame group for a single patient,
                              containing all their visits.

    Returns:
        int: The count of unique physicians associated with the patient.
    """
    # Stack all physician columns into a single Series, drop NaNs, and get unique values.
    physicians = group[['Primary_Physician', 'Operating_Physician', 'Other_Physician']].stack().unique()
    return len(physicians)

In [79]:
total_physicians = df.groupby('Patient_ID').apply(unique_physicians).reset_index(name='Total_Physicians')

C:\Users\Mustafa Khan\AppData\Local\Temp\ipykernel_24104\2735038221.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  total_physicians = df.groupby('Patient_ID').apply(unique_physicians).reset_index(name='Total_Physicians')


### Get Unique Diagnoses for each Patient

In [80]:
def unique_diagnoses(group):
    """
    Counts the number of unique diagnoses for a given patient group.
    Combines all ICD9_DGNS_CD columns, filters out NaN values,
    and returns the count of unique diagnoses.
    """
    # Select all columns that represent diagnoses.
    diagnosis_cols = [col for col in group.columns if 'ICD9_DGNS_CD' in col]
    # Flatten all diagnosis codes into a single Series for the patient, drop NaNs, and get unique values.
    all_diagnoses = group[diagnosis_cols].stack().unique()
    return len(all_diagnoses)

In [81]:
# Apply the function to each patient group to get the total unique diagnoses.
total_diagnoses = df.groupby('Patient_ID').apply(unique_diagnoses).reset_index(name='Total_Diagnosis')

C:\Users\Mustafa Khan\AppData\Local\Temp\ipykernel_24104\198543127.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  total_diagnoses = df.groupby('Patient_ID').apply(unique_diagnoses).reset_index(name='Total_Diagnosis')


### Get Most Frequent Diagnosis for each Patient

In [82]:
def most_frequent_diagnosis(group):
    """
    Identifies the most frequent diagnosis for a group of patient visits.
    In case of ties, it picks one at random.
    """
    diagnosis_cols = [col for col in group.columns if 'ICD9_DGNS_CD' in col]
    all_diagnoses = group[diagnosis_cols].values.flatten()
    
    # Filter out NaN values before counting
    remove_nan_diagnoses = pd.Series(all_diagnoses).dropna()
    
    if remove_nan_diagnoses.empty:
        # Return NaN if no diagnoses are present
        return np.nan 
    else:
        return remove_nan_diagnoses.mode()[0] 

In [83]:
# Apply the function to each patient group to get the most frequent diagnoses.
most_frequent_diagnosis = df.groupby('Patient_ID').apply(most_frequent_diagnosis).reset_index(name='Most_Freq_Diagnosis')

C:\Users\Mustafa Khan\AppData\Local\Temp\ipykernel_24104\117985977.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  most_frequent_diagnosis = df.groupby('Patient_ID').apply(most_frequent_diagnosis).reset_index(name='Most_Freq_Diagnosis')


### Merge all data 

In [84]:
summary_df = total_visits.merge(total_physicians, on='Patient_ID')
summary_df = summary_df.merge(total_diagnoses, on='Patient_ID')
summary_df = summary_df.merge(most_frequent_diagnosis, on='Patient_ID')

### FINAL RESULT

In [85]:
print(summary_df.head())

         Patient_ID  Total_Visits  Total_Physicians  Total_Diagnosis  \
0  00013D2EFD8E45D1             1                 1                1   
1  00016F745862898F             2                 2                9   
2  0001FDD721E223DC             1                 1                3   
3  00024B3D2352D2D0             4                 4                4   
4  0002F28CE057345B            19                21               45   

  Most_Freq_Diagnosis  
0               V5841  
1                2724  
2               78943  
3                2723  
4                3569  
